<a href="https://colab.research.google.com/github/amzad-786githumb/SQLite_for_Python_and_R/blob/main/python_sql_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Library to write SQL queries</h1>

<p>
python-sql is a library to write SQL queries in a pythonic way.</p>

In [1]:
pip install python-sql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 3.0 MB/s eta 0:00:00


<h2>Import:</h2>

In [2]:
from sql import *
from sql.aggregate import *
from sql.conditionals import *

<h3>Simple selects:</h3>

In [3]:
user = Table('user')
select = user.select()
tuple(select)
('SELECT * FROM "user" AS "a"', ())

('SELECT * FROM "user" AS "a"', ())

In [4]:
select = user.select(user.name)
tuple(select)
('SELECT "a"."name" FROM "user" AS "a"', ())

('SELECT "a"."name" FROM "user" AS "a"', ())

In [5]:
select = user.select(Count(Literal(1)))
tuple(select)
('SELECT COUNT(%s) FROM "user" AS "a"', (1,))

('SELECT COUNT(%s) FROM "user" AS "a"', (1,))

In [6]:
select = user.select(user.name, distinct=True)
tuple(select)
('SELECT DISTINCT "a"."name" FROM "user" AS "a"', ())

('SELECT DISTINCT "a"."name" FROM "user" AS "a"', ())

In [7]:
select = user.select(user.id, user.name)
tuple(select)
('SELECT "a"."id", "a"."name" FROM "user" AS "a"', ())

('SELECT "a"."id", "a"."name" FROM "user" AS "a"', ())

<h3>Select with where condition:</h3>

In [8]:
select.where = user.name == 'foo'
tuple(select)
('SELECT "a"."id", "a"."name" FROM "user" AS "a" WHERE ("a"."name" = %s)', ('foo',))

('SELECT "a"."id", "a"."name" FROM "user" AS "a" WHERE ("a"."name" = %s)',
 ('foo',))

In [10]:
select.where = (user.name == 'foo') & (user.active == True)
tuple(select)
('SELECT "a"."id", "a"."name" FROM "user" AS "a" WHERE (("a"."name" = %s) AND ("a"."active" = %s))', ('foo', True))
select.where = user.name == user.login
tuple(select)
('SELECT "a"."id", "a"."name" FROM "user" AS "a" WHERE ("a"."name" = "a"."login")', ())

('SELECT "a"."id", "a"."name" FROM "user" AS "a" WHERE ("a"."name" = "a"."login")',
 ())

('SELECT "a"."id", "a"."name" FROM "user" AS "a" WHERE ("a"."name" = "a"."login")',
 ())

<h3>Select with join:</h3>

In [11]:
join = user.join(Table('user_group'))
join.condition = join.right.user == user.id
select = join.select(user.name, join.right.group)
tuple(select)
('SELECT "a"."name", "b"."group" FROM "user" AS "a" INNER JOIN "user_group" AS "b" ON ("b"."user" = "a"."id")', ())

('SELECT "a"."name", "b"."group" FROM "user" AS "a" INNER JOIN "user_group" AS "b" ON ("b"."user" = "a"."id")',
 ())

<h3>Select with multiple joins:</h3>

In [12]:
join1 = user.join(Table('user'))
join2 = join1.join(Table('user'))
select = join2.select(user.id, join1.right.id, join2.right.id)
tuple(select)
('SELECT "a"."id", "b"."id", "c"."id" FROM "user" AS "a" INNER JOIN "user" AS "b" INNER JOIN "user" AS "c"', ())

('SELECT "a"."id", "b"."id", "c"."id" FROM "user" AS "a" INNER JOIN "user" AS "b" INNER JOIN "user" AS "c"',
 ())

<h3>Select with group_by:</h3>

In [14]:
invoice = Table('invoice')
select = invoice.select(Sum(invoice.amount), invoice.currency, group_by=invoice.currency)
tuple(select)
('SELECT SUM("a"."amount"), "a"."currency" FROM "invoice" AS "a" GROUP BY "a"."currency"', ())

('SELECT SUM("a"."amount"), "a"."currency" FROM "invoice" AS "a" GROUP BY "a"."currency"',
 ())

<h3>Select with output name:</h3>

In [15]:
tuple(user.select(user.name.as_('First Name')))
('SELECT "a"."name" AS "First Name" FROM "user" AS "a"', ())

('SELECT "a"."name" AS "First Name" FROM "user" AS "a"', ())

<h3>Select with order_by:</h3>

In [16]:
tuple(user.select(order_by=user.date))
('SELECT * FROM "user" AS "a" ORDER BY "a"."date"', ())
tuple(user.select(order_by=Asc(user.date)))
('SELECT * FROM "user" AS "a" ORDER BY "a"."date" ASC', ())
tuple(user.select(order_by=(user.date.asc, user.id.desc)))
('SELECT * FROM "user" AS "a" ORDER BY "a"."date" ASC, "a"."id" DESC', ())

('SELECT * FROM "user" AS "a" ORDER BY "a"."date" ASC, "a"."id" DESC', ())

<h3>Select with sub-select:</h3>

In [17]:
user_group = Table('user_group')
subselect = user_group.select(user_group.user, where=user_group.active == True)
user = Table('user')
tuple(user.select(user.id, where=user.id.in_(subselect)))
('SELECT "a"."id" FROM "user" AS "a" WHERE ("a"."id" IN (SELECT "b"."user" FROM "user_group" AS "b" WHERE ("b"."active" = %s)))', (True,))
tuple(subselect.select(subselect.user))
('SELECT "a"."user" FROM (SELECT "b"."user" FROM "user_group" AS "b" WHERE ("b"."active" = %s)) AS "a"', (True,))

('SELECT "a"."user" FROM (SELECT "b"."user" FROM "user_group" AS "b" WHERE ("b"."active" = %s)) AS "a"',
 (True,))

<h3>Select on other schema:</h3>

In [18]:
other_table = Table('user', 'myschema')
tuple(other_table.select())
('SELECT * FROM "myschema"."user" AS "a"', ())

('SELECT * FROM "myschema"."user" AS "a"', ())

<h3>Insert query with default values:</h3>

In [20]:
tuple(user.insert())
('INSERT INTO "user" AS "a" DEFAULT VALUES', ())

('INSERT INTO "user" AS "a" DEFAULT VALUES', ())

<h3>Insert query with values:</h3>

In [21]:
tuple(user.insert(columns=[user.name, user.login], values=[['Foo', 'foo']]))
('INSERT INTO "user" AS "a" ("name", "login") VALUES (%s, %s)', ('Foo', 'foo'))
tuple(user.insert(columns=[user.name, user.login], values=[['Foo', 'foo'], ['Bar', 'bar']]))
('INSERT INTO "user" AS "a" ("name", "login") VALUES (%s, %s), (%s, %s)', ('Foo', 'foo', 'Bar', 'bar'))

('INSERT INTO "user" AS "a" ("name", "login") VALUES (%s, %s), (%s, %s)',
 ('Foo', 'foo', 'Bar', 'bar'))

<h3>Insert query with query:</h3>

In [22]:
passwd = Table('passwd')
select = passwd.select(passwd.login, passwd.passwd)
tuple(user.insert(values=select))
('INSERT INTO "user" AS "b" SELECT "a"."login", "a"."passwd" FROM "passwd" AS "a"', ())

('INSERT INTO "user" AS "b" SELECT "a"."login", "a"."passwd" FROM "passwd" AS "a"',
 ())

<h3>Update query with values:</h3>

In [23]:
tuple(user.update(columns=[user.active], values=[True]))
('UPDATE "user" AS "a" SET "active" = %s', (True,))
tuple(invoice.update(columns=[invoice.total], values=[invoice.amount + invoice.tax]))
('UPDATE "invoice" AS "a" SET "total" = ("a"."amount" + "a"."tax")', ())

('UPDATE "invoice" AS "a" SET "total" = ("a"."amount" + "a"."tax")', ())

<h3>Update query with where condition:</h3>

In [24]:
tuple(user.update(columns=[user.active], values=[True], where=user.active == False))
('UPDATE "user" AS "a" SET "active" = %s WHERE ("a"."active" = %s)', (True, False))

('UPDATE "user" AS "a" SET "active" = %s WHERE ("a"."active" = %s)',
 (True, False))

<h3>Update query with from list:</h3>

In [25]:
group = Table('user_group')
tuple(user.update(columns=[user.active], values=[group.active],
from_=[group], where=user.id == group.user))
('UPDATE "user" AS "b" SET "active" = "a"."active" FROM "user_group" AS "a" WHERE ("b"."id" = "a"."user")', ())

('UPDATE "user" AS "b" SET "active" = "a"."active" FROM "user_group" AS "a" WHERE ("b"."id" = "a"."user")',
 ())

<h3>Delete query:</h3>

In [26]:
tuple(user.delete())
('DELETE FROM "user"', ())

('DELETE FROM "user"', ())

<h3>Delete query with where condition:</h3>

In [27]:
tuple(user.delete(where=user.name == 'foo'))
('DELETE FROM "user" WHERE ("name" = %s)', ('foo',))

('DELETE FROM "user" WHERE ("name" = %s)', ('foo',))

<h3>Delete query with sub-query:</h3>

In [28]:
tuple(user.delete(where=user.id.in_(user_group.select(user_group.user))))
('DELETE FROM "user" WHERE ("id" IN (SELECT "a"."user" FROM "user_group" AS "a"))', ())

('DELETE FROM "user" WHERE ("id" IN (SELECT "a"."user" FROM "user_group" AS "a"))',
 ())

<h3>Flavors:</h3>

In [29]:
select = user.select()
select.offset = 10
Flavor.set(Flavor())
tuple(select)
('SELECT * FROM "user" AS "a" OFFSET 10', ())
Flavor.set(Flavor(max_limit=18446744073709551615))
tuple(select)
('SELECT * FROM "user" AS "a" LIMIT 18446744073709551615 OFFSET 10', ())
Flavor.set(Flavor(max_limit=-1))
tuple(select)
('SELECT * FROM "user" AS "a" LIMIT -1 OFFSET 10', ())

('SELECT * FROM "user" AS "a" LIMIT -1 OFFSET 10', ())

<h3>Limit style:</h3>

In [30]:
select = user.select(limit=10, offset=20)
Flavor.set(Flavor(limitstyle='limit'))
tuple(select)
('SELECT * FROM "user" AS "a" LIMIT 10 OFFSET 20', ())
Flavor.set(Flavor(limitstyle='fetch'))
tuple(select)
('SELECT * FROM "user" AS "a" OFFSET (20) ROWS FETCH FIRST (10) ROWS ONLY', ())
Flavor.set(Flavor(limitstyle='rownum'))
tuple(select)
('SELECT "a".* FROM (SELECT "b".*, ROWNUM AS "rnum" FROM (SELECT * FROM "user" AS "c") AS "b" WHERE (ROWNUM <= %s)) AS "a" WHERE ("rnum" > %s)', (30, 20))

('SELECT "a".* FROM (SELECT "b".*, ROWNUM AS "rnum" FROM (SELECT * FROM "user" AS "c") AS "b" WHERE (ROWNUM <= %s)) AS "a" WHERE ("rnum" > %s)',
 (30, 20))

<h3>qmark style:</h3>

In [31]:
Flavor.set(Flavor(paramstyle='qmark'))
select = user.select()
select.where = user.name == 'foo'
tuple(select)
('SELECT * FROM "user" AS "a" WHERE ("a"."name" = ?)', ('foo',))

('SELECT * FROM "user" AS "a" WHERE ("a"."name" = ?)', ('foo',))

<h3>numeric style:</h3>

In [33]:
Flavor.set(Flavor(paramstyle='format'))
select = user.select()
select.where = user.name == 'foo'
format2numeric(*select)
('SELECT * FROM "user" AS "a" WHERE ("a"."name" = :0)', ('foo',))

('SELECT * FROM "user" AS "a" WHERE ("a"."name" = :0)', ('foo',))